In [1]:
import gymnasium as gym

from envs.rllib import DummyEnvironment
from src.wrappers.rllib import UniformlyRestrictedEnvironment
from src.utils import test_environment

In [2]:
test_environment(DummyEnvironment)

Testing <class 'envs.rllib.DummyEnvironment'>...
Reset: {'0': array([0.6727912, 0.8889548], dtype=float32), '1': array([0.55685323, 0.75645125], dtype=float32)}
Actions: {'0': 2, '1': 2}
Step:  {'0': array([0.53469455, 0.94462395], dtype=float32), '1': array([0.86224824, 0.60816336], dtype=float32)}, {'0': 0.77840954776388, '1': 0.21310117729227895}, {'__all__': False}
Actions: {'0': 2, '1': 1}
Step:  {'0': array([0.18398279, 0.41683522], dtype=float32), '1': array([0.76554835, 0.3123363 ], dtype=float32)}, {'0': 0.07815280993777385, '1': 0.5088450271694417}, {'__all__': False}
Actions: {'0': 2, '1': 0}
Step:  {'0': array([0.839865 , 0.7542257], dtype=float32), '1': array([0.44633624, 0.78522897], dtype=float32)}, {'0': 0.4750707449221223, '1': 0.15724584745842396}, {'__all__': False}
Actions: {'0': 2, '1': 0}
Step:  {'0': array([0.484708  , 0.33523908], dtype=float32), '1': array([0.5028542 , 0.17261794], dtype=float32)}, {'0': 0.622444759916867, '1': 0.7606707022044892}, {'__all__': 

In [3]:
env_config = { 'action_space': gym.spaces.Box(0.0, 1.0)}

test_environment(UniformlyRestrictedEnvironment({ 'env': DummyEnvironment, 'env_config': env_config, 'governance_action_space': gym.spaces.Box(0.0, 1.0)}))

Testing <class 'src.wrappers.rllib.UniformlyRestrictedEnvironment'>...
Reset: {'gov': 0}
Actions: {'gov': array([0.44632417], dtype=float32)}
Step:  {'0': {'observation': array([0.1120354 , 0.85574687], dtype=float32), 'allowed_actions': array([0.44632417], dtype=float32)}, '1': {'observation': array([0.38524696, 0.9515381 ], dtype=float32), 'allowed_actions': array([0.44632417], dtype=float32)}}, {}, {'__all__': False}
Actions: {'0': array([0.30087507], dtype=float32), '1': array([0.9527573], dtype=float32)}
Step:  {'gov': 0}, {'gov': 1.0436594376465638}, {'__all__': False}
Actions: {'gov': array([0.35408455], dtype=float32)}
Step:  {'0': {'observation': array([0.37391806, 0.46113598], dtype=float32), 'allowed_actions': array([0.35408455], dtype=float32)}, '1': {'observation': array([0.16022688, 0.51763   ], dtype=float32), 'allowed_actions': array([0.35408455], dtype=float32)}}, {'0': 0.17782235825321835, '1': 0.8658370793933455}, {'__all__': False}
Actions: {'0': array([0.33343732],